Using the test_spread.csv.gz find the following values;
1.	Mean Spread
2.	Time Weighted Average Spread
3.	Volume Weighted Average Spread
4.	Volatility of the mid price (using a sampling frequency of 1 minute)

Using the test_trades.csv find the following values;
5.	VWAP for the day for LIT bmll_trade_type trades only
6.	VWAP between 14:00 - 15:00 for LIT bmll_trade_type trades only
7.	Average notional per bmll_trade_type

All work must be done using a Python Jupyter Lab/Notebook with results clearly identified in the code comments/notation, and in the email text.

All notional values to be in Euro

The test itself shouldn't take more than 2 hours, the deadline for return of completed work is Friday1200. 


In [1]:
import pandas as pd

# file path for the CSV file
csv_file_path = 'https://raw.githubusercontent.com/datahub83/BMLL/main/test_spread.csv'

# Read the CSV file into a pandas DataFrame
df = pd.read_csv(csv_file_path)

# Display the DataFrame
print(df)

                      event_timestamp  best_bid_size  best_bid_price  \
0       2023-11-02 08:00:18.477458987            5.0           75.26   
1       2023-11-02 08:00:18.484837247            5.0           75.26   
2       2023-11-02 08:00:18.504462667            5.0           75.26   
3       2023-11-02 08:00:18.526210087          395.0           75.26   
4       2023-11-02 08:00:18.526292987          395.0           75.26   
...                               ...            ...             ...   
116592  2023-11-02 16:30:00.017715246         6160.0           77.13   
116593  2023-11-02 16:30:00.018655506        12044.0           77.13   
116594  2023-11-02 16:30:00.018715906        12044.0           77.13   
116595  2023-11-02 16:30:00.029214066        12044.0           77.13   
116596  2023-11-02 16:30:00.121807526            NaN             NaN   

        best_ask_price  best_ask_size        market_state  
0                75.41         5776.0  CONTINUOUS_TRADING  
1              

FIRST PART: Using the test_spread.csv.gz find the following values;
1.	Mean Spread
2.	Time Weighted Average Spread
3.	Volume Weighted Average Spread
4.	Volatility of the mid price (using a sampling frequency of 1 minute)

In [2]:
# 1.	Mean Spread

df['spread'] = df['best_ask_price'] - df['best_bid_price']

# Compute the mean spread
mean_spread = df['spread'].mean()

print("Mean Spread:", mean_spread)

Mean Spread: 0.031973909911146084


In [3]:
# 2.	Time Weighted Average Spread

df['event_timestamp'] = pd.to_datetime(df['event_timestamp'])

# Sort DataFrame by 'event_timestamp' in ascending order
df = df.sort_values(by='event_timestamp')

# Calculate time differences between consecutive rows (in seconds)
df['time_diff'] = df['event_timestamp'].diff().dt.total_seconds().fillna(0)

# Calculate spread for each row
df['spread'] = df['best_ask_price'] - df['best_bid_price']

# Calculate spread * time_diff for each row
df['spread_time'] = df['spread'] * df['time_diff']

# Calculate the total time duration
total_time = df['time_diff'].sum()

# Calculate the Time Weighted Average Spread (TWAS)
twas = (df['spread_time'].sum()) / total_time

print("Time Weighted Average Spread (TWAS):", twas)

Time Weighted Average Spread (TWAS): 0.03205626435887015


In [4]:
# 3.	Volume Weighted Average Spread

df['event_timestamp'] = pd.to_datetime(df['event_timestamp'])

# Sort DataFrame by 'event_timestamp' in ascending order
df = df.sort_values(by='event_timestamp')

# Calculate time differences between consecutive rows (in seconds)
df['time_diff'] = df['event_timestamp'].diff().dt.total_seconds().fillna(0)

# Calculate spread for each row
df['spread'] = df['best_ask_price'] - df['best_bid_price']

# Calculate the spread * trading volume for each row
df['spread_volume'] = df['spread'] * df['best_ask_size']

# Calculate the total trading volume
total_volume = df['best_ask_size'].sum()

# Calculate the Volume Weighted Average Spread (VWAS)
vwas = (df['spread_volume'].sum()) / total_volume

print("Volume Weighted Average Spread (VWAS):", vwas)

Volume Weighted Average Spread (VWAS): 0.03292758699113055


In [5]:
#4.	Volatility of the mid price (using a sampling frequency of 1 minute)

df['event_timestamp'] = pd.to_datetime(df['event_timestamp'])

# Set 'event_timestamp' as the index
df.set_index('event_timestamp', inplace=True)

# Resample the DataFrame to 1-minute intervals and calculate the mid price
mid_price = (df['best_bid_price'] + df['best_ask_price']) / 2
mid_price_resampled = mid_price.resample('1min').mean()

# Calculate the log returns of the mid price
log_returns = mid_price_resampled.diff().dropna()

# Calculate the standard deviation of the log returns (volatility)
volatility = log_returns.std()

print("Volatility of the mid price (1-minute sampling frequency):", volatility)

Volatility of the mid price (1-minute sampling frequency): 0.04365133620574938


SECOND PART: Using the test_trades.csv find the following values;
5.	VWAP for the day for LIT bmll_trade_type trades only
6.	VWAP between 14:00 - 15:00 for LIT bmll_trade_type trades only
7.	Average notional per bmll_trade_type

In [6]:
import pandas as pd

# file path for the CSV file
csv_file_path2 = 'https://raw.githubusercontent.com/datahub83/BMLL/main/test_trades.csv'

# Read the CSV file into a pandas DataFrame
df = pd.read_csv(csv_file_path2)

# Display the DataFrame
print(df)

                 trade_id                trade_timestamp  \
0     1034002182352621680  2023-11-01 17:32:36.702000000   
1         873382216205192  2023-11-02 08:00:18.473746000   
2         873382216205267  2023-11-02 08:00:18.850717000   
3         873382216205268  2023-11-02 08:00:18.851257000   
4         873382216205269  2023-11-02 08:00:18.851257000   
...                   ...                            ...   
9317   613331775096836208  2023-11-02 17:57:19.000000000   
9318   462813306178719856  2023-11-02 17:57:23.000000000   
9319  1096273978486116464  2023-11-02 18:00:08.000000000   
9320   323012670987653232  2023-11-02 18:00:16.000000000   
9321   665071017293205616  2023-11-02 18:45:11.000000000   

              publication_timestamp aggressor_side     price  execution_size  \
0     2023-11-02 07:15:23.856742733        UNKNOWN  75.13000           40000   
1     2023-11-02 08:00:18.477383475        UNKNOWN  75.40000          158559   
2     2023-11-02 08:00:18.850782887    

In [7]:
#5.	VWAP for the day for LIT bmll_trade_type trades only

# Filter the DataFrame for LIT bmll_trade_type trades
lit_trades = df[df['bmll_trade_type'] == 'LIT'].copy()

# Convert currency from GBp to EUR based on the provided exchange rates
exchange_rates = pd.read_csv('https://raw.githubusercontent.com/datahub83/BMLL/main/fx_rates.csv') 

mean_exchange_rate = exchange_rates['GBpEUR'].mean()
lit_trades.loc[:, 'price'] *= mean_exchange_rate

# Calculate trade value for each trade
lit_trades.loc[:, 'trade_value'] = lit_trades['price'] * lit_trades['execution_size']

# Calculate cumulative trade value and cumulative trade volume
lit_trades.loc[:, 'cumulative_trade_value'] = lit_trades['trade_value'].cumsum()
lit_trades.loc[:, 'cumulative_trade_volume'] = lit_trades['execution_size'].cumsum()

# Calculate VWAP
vwap = lit_trades['cumulative_trade_value'].iloc[-1] / lit_trades['cumulative_trade_volume'].iloc[0]

print("Volume Weighted Average Price (VWAP) for the day for LIT bmll_trade_type trades:", vwap)

Volume Weighted Average Price (VWAP) for the day for LIT bmll_trade_type trades: 4964.148741862237


In [8]:
# 6. VWAP between 14:00 - 15:00 for LIT bmll_trade_type trades only

# Convert trade_timestamp to datetime format
df['trade_timestamp'] = pd.to_datetime(df['trade_timestamp'])

# Filter the DataFrame to include only LIT bmll_trade_type trades
lit_trades = df[df['bmll_trade_type'] == 'LIT']

# Filter the DataFrame to include trades between 14:00 and 15:00
lit_trades = lit_trades[(lit_trades['trade_timestamp'].dt.hour == 14) & 
                        (lit_trades['trade_timestamp'].dt.minute >= 0) & 
                        (lit_trades['trade_timestamp'].dt.minute <= 59)]

# Calculate trade value for each trade
lit_trades['trade_value'] = lit_trades['price'] * lit_trades['execution_size']

# Calculate cumulative trade value and cumulative trade volume
lit_trades['cumulative_trade_value'] = lit_trades['trade_value'].cumsum()
lit_trades['cumulative_trade_volume'] = lit_trades['execution_size'].cumsum()

# Calculate VWAP
vwap = lit_trades['cumulative_trade_value'].iloc[-1] / lit_trades['cumulative_trade_volume'].iloc[-1]

print("Volume Weighted Average Price (VWAP) between 14:00 and 15:00 for LIT bmll_trade_type trades:", vwap)

Volume Weighted Average Price (VWAP) between 14:00 and 15:00 for LIT bmll_trade_type trades: 76.98892052319027


In [9]:
#7.	Average notional per bmll_trade_type

# Calculate the notional value for each trade
df['notional'] = df['price'] * df['execution_size']

# Group the DataFrame by bmll_trade_type and calculate the mean notional for each group
average_notional_per_type = df.groupby('bmll_trade_type')['notional'].mean()

# Display the average notional per bmll_trade_type
print("Average notional per bmll_trade_type:")
print(average_notional_per_type)

Average notional per bmll_trade_type:
bmll_trade_type
BENCHMARK_PRICE    4.408146e+07
DARK               6.044831e+05
LIT                2.278095e+05
OTC                2.133418e+05
SPECIAL_PRICE      1.812250e+05
UNCROSSING         8.465001e+08
Name: notional, dtype: float64
